In [1]:
#Importing the necessary libraries
import numpy as np
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

C:\Users\binur\anaconda3\envs\tfradeon\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\binur\anaconda3\envs\tfradeon\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Loading the data
bc = datasets.load_breast_cancer()
x = bc.data
y = bc.target

In [3]:
n_samples,n_features = x.shape

In [4]:
#Splitting the dataset into training and testing
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1234)

In [5]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

* These lines of code are normalizing the features of the training and test sets.


* The StandardScaler class from scikit-learn is being instantiated, and then fit_transform method is used to fit the scaler to the training data and return the transformed training data.


* **This step is done to standardize the data by removing the mean and scaling it to unit variance.**


* The same scaling parameters are then applied to the test set by calling the transform method. 


* This is an important step to ensure that the test data is transformed in the same way as the training data, which is necessary to make fair and accurate predictions.

In [7]:
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

* The **from_numpy()** method from the torch library is used to convert the **numpy array x_train to a PyTorch tensor**.


* The **astype()** method from NumPy is used to change the **data type of the array from its original type to a float32 data type.**


* This step is necessary because some of the PyTorch functions that may be used later in the code expect the data to be in a specific format, such as a float32 tensor.

In [8]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

* **def __init__(self, n_input_features)** is a special method in Python classes known as a **constructor**. It is automatically called when an object of the class is created.


* In the __init__ method, self refers to the object that the method is being called on. In other words, self refers to the current instance of the class.


* When an object of the class is created and the __init__ method is called, the object itself is passed as the first argument to the method, and that object is assigned to the self parameter.


* In this specific case, the self parameter is used to create an attribute of the class called linear and assigns it the value of a linear layer created using the nn.Linear class from PyTorch, with the number of inputs as n_input_features and 1 output.


* In the forward method, self is used to refer to the linear attribute of the current instance of the class, so that the method can apply the linear layer to the input tensor and return the result.


* In short, self allows the class methods to access and modify the attributes of the current instance of the class, and it is necessary for the class to work as expected.


* In this specific case, the constructor is used to initialize the model's parameters when an object of the class Model is created. 


* The constructor takes in a single argument n_input_features which is passed to the linear layer, so the layer will have n_input_features inputs and one output.


* The super(Model,self).__init__() is calling the constructor of the parent class nn.Module, this is necessary to ensure that the parent class's initialization is also done.


* Overall, the __init__ method is used to initialize the initial state of an object, by setting the initial values of the object's attributes.

In [9]:
#Building the model class
class Model(nn.Module):
    def __init__(self,n_input_features):
        super(Model,self).__init__()
        self.linear = nn.Linear(n_input_features,1)
        
    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

* This code defines a PyTorch model class called "Model" that inherits from the nn.Module class from the PyTorch neural network library.


* The class has an init method, a constructor that is automatically called when an object of the class is created. 


* The constructor initializes the model's parameters, in this case, it creates a single linear layer with n_input_features inputs and 1 output.


* The class also has a forward method, which takes in an input tensor x and applies the linear layer on it, then it applies the sigmoid function on the output of the linear layer.


* The sigmoid function maps the output of the linear layer to a value between 0 and 1, this can be used to represent the probability of the input belonging to a certain class.


* The forward method returns the output of the sigmoid function, which can be interpreted as the model's predicted probability of the input belonging to the positive class.

In [10]:
model = Model(n_features)

* This line of code creates an object of the Model class and assigns it to the variable model.


* The number of features of the input data is passed as an argument to the class constructor (__init__ method) when creating the object.


* This step is necessary because the constructor of the Model class expects an argument which is the number of features of the input data and uses it to initialize the linear layer of the model with the appropriate number of inputs.


* Once the object is created, it is ready to be used for training and making predictions.


* It's worth noting that creating an object of the class does not perform any computation, it only initializes the model's parameters in this case a single linear layer with the number of inputs as the number of features provided.

* nn.BCELoss() is a function from the PyTorch library that creates an instance of the Binary Cross-Entropy Loss (BCE Loss) criterion.


* The BCE Loss is a loss function commonly used for training models for binary classification tasks. 


* It calculates the difference between the predicted probability of the positive class and the true label, and it is commonly used in combination with the sigmoid activation function.


* The BCE Loss calculates the loss by computing the cross-entropy between the predicted probability and the true labels.


* The predicted probability is given by the sigmoid function of the model's output, and the true labels are binary (0 or 1).


* The BCE Loss function is defined as: **-loss(x, class) = -(x[class] + log(sum(exp(x))))**


* The BCE Loss should be used in combination with an optimizer to update the model's parameters during training.

* **torch.optim.SGD()** creates an instance of the **Stochastic Gradient Descent(SGD) optimizer** from the PyTorch library.

In [11]:
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [12]:
#Training
for epoch in range(num_epochs):
    #Forward pass and loss
    y_pred = model(x_train)
    loss = criterion(y_pred,y_train)
    
    #Backward pass and update
    loss.backward()
    optimizer.step()
    
    #Zero grad before new step
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.6118
epoch: 20, loss = 0.4983
epoch: 30, loss = 0.4270
epoch: 40, loss = 0.3781
epoch: 50, loss = 0.3425
epoch: 60, loss = 0.3152
epoch: 70, loss = 0.2936
epoch: 80, loss = 0.2760
epoch: 90, loss = 0.2612
epoch: 100, loss = 0.2487


* This code defines a for loop that runs for a specified number of epochs (num_epochs).

* An epoch is one complete iteration over the entire training dataset.

* In each iteration of the loop, the following steps are performed:
1. The forward pass: the input tensor x_train is passed through the model, and the model's output y_pred is obtained.


2. The loss calculation: the BCE Loss criterion is applied to the model's output y_pred and the true labels y_train, and the value of the loss is obtained.


3. The backward pass: the gradients of the loss with respect to the model's parameters are calculated using the backward() method.


4. The model's parameters are updated: the step() method of the optimizer is called to update the model's parameters based on the gradients calculated in the previous step.


5. The gradients are zeroed before the new step: the zero_grad() method of the optimizer is called to clear the gradients from the previous step.


6. Print the current loss after every 10th iteration: The current loss is printed out every 10th iteration of the loop, this is useful for monitoring the progress of the training process and to make sure that the model is learning.


* **loss.item()** function is to extract the scalar value from the tensor, this is useful when we want to print the value of the loss, or to check the value of the loss and compare it to a threshold to stop the training.

In [13]:
with torch.no_grad():
    y_predicted = model(x_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy:{acc.item():.4f}')

accuracy:0.9211


* This code is evaluating the trained model on the test dataset.


* It creates a context using torch.no_grad() which is used to evaluate the model and make predictions on the test set.


* Inside this context, the following steps are performed:
1. The forward pass: the input tensor X_test is passed through the model, and the model's output y_predicted is obtained.


2. The predictions are rounded: the round() function is used to round the predicted probabilities to 0 or 1. This step is used to convert the predicted probabilities to binary class labels, which can be compared to the true labels.


3. Accuracy is calculated: the accuracy is calculated by comparing the rounded predicted labels y_predicted_cls to the true labels y_test and counting the number of correct predictions. The accuracy is then calculated as the ratio of the number of correct predictions to the total number of samples in the test set.


4. The accuracy is printed: The accuracy is printed out to evaluate the performance of the model.


* **torch.no_grad()** is used to prevent the autograd engine from tracking the gradients during the forward pass, this is useful when we are only interested in the forward pass and do not want to use the gradients, this can save memory and computation.


* The **eq()** function is a PyTorch function that compares two tensors element-wise and returns a new tensor with the same shape as the input tensors, but with binary values (0 or 1) indicating whether the corresponding elements of the input tensors are equal or not.


* In the code you provided, the eq() function is used to compare the rounded predicted labels y_predicted_cls to the true labels y_test. It returns a new tensor of the same shape as the input tensors, but with binary values indicating whether the corresponding predicted label and true label are equal or not.


* For example, if the predicted label is 1 and the true label is also 1, the corresponding element in the output tensor will be 1, indicating that the prediction was correct. If the predicted label is 0 and the true label is 1, the corresponding element in the output tensor will be 0, indicating that the prediction was incorrect.


* In summary, the eq() function compares the predicted labels and true labels element-wise, and returns a tensor indicating whether the predictions are correct or not, and then the accuracy is calculated by counting the number of correct predictions and dividing it by the total number of predictions.